In this notebook, we want to increase the complexity of the transfer learning for the model SAM from meta. Inthe previous notebook, we tested a lean version of transfer learning with SAM. 

* 100 epochs
* 10 patience
* Pytorch only instead of mixing HuggingFace with PyTorch
    * It seems like it's hard to rely only on PyTorch

In [3]:
# Setting up the enviroment

%pip install -q transformers
%pip install -q datasets
%pip install -q Pillow
%pip install -q torch
%pip install -q torchmetrics
%pip install -q tdqm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

cwd = os.getcwd()
print(cwd)

/home/mark/work/MultiModal


In [13]:
from datasets import load_dataset, Image
data_dir = os.path.join(cwd, 'data')

one_class_flipped_dataset = "sentinel-1-flipped-one-class/train/"
two_class_augmented_dataset = "sentinel-1-augmented-two-class/train/"

current_data_dir = os.path.join(data_dir, one_class_flipped_dataset)
current_data_dir 

'/home/mark/work/MultiModal/data/sentinel-1-flipped-one-class/train/'

In [51]:
# Load and split the dataset 

ds = load_dataset(path = current_data_dir, split='train')
ds = ds.train_test_split(test_size=0.2, seed=42)
ds



DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 320
    })
    test: Dataset({
        features: ['image'],
        num_rows: 80
    })
})

In [ ]:
# Check the classes (It can be more than 1, depending on the dataset)
import pandas as pd

csv_path = os.path.join(current_data_dir, '_classes.csv')

print(csv_path)

df = pd.read_csv(csv_path)
df

/home/mark/work/MultiModal/data/sentinel-1-flipped-one-class/train/_classes.csv


,Pixel Value,Class
0,0,background
1,1,water


In [53]:
# Still data exploration
sample = ds['train'][3]
sample 

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=250x140>}

In [54]:
sample_image = sample['image']
sample_image.show()